In [1]:
from Utils import *

# Step 13
in this step we prepare data for Deep Learning Model

2) convert TTC feature to a vector only with TTC values


### Load data from folder: "10_data_with_ttc_and_distance_2_zebra"

In [2]:
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"

In [3]:
directory = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\12_data_regions_feature'
# Get a list of files in the directory
file_name_list = os.listdir(directory)
print(file_name_list)
recordingId_list=get_recordingsId_list(file_name_list)
recordingId_list

['18_data_with_zones.csv', '19_data_with_zones.csv', '20_data_with_zones.csv', '21_data_with_zones.csv', '22_data_with_zones.csv', '23_data_with_zones.csv', '24_data_with_zones.csv', '25_data_with_zones.csv', '26_data_with_zones.csv', '27_data_with_zones.csv', '28_data_with_zones.csv', '29_data_with_zones.csv']


['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [4]:
dfs = load_dfs(file_name_list, root_dir, directory)

In [5]:
folder_name="13_data_tcc_flattened"
create_folder_to_save_data(root_dir, folder_name)

Folder: '13_data_tcc_flattened' already exists


### find max of pair numbers:

In [6]:
max_value = -float('inf')
max_df_names = []

for fname, df in zip(file_name_list, dfs):
    df_max = df['num_of_pairs'].max()
    if df_max > max_value:
        max_value = df_max
        max_df_names = [fname]
    elif df_max == max_value:
        max_df_names.append(fname)

print(f"Maximum value in 'num_of_pairs' across all dfs:  {max_value}")
print(f"Found in files:                                  {max_df_names}")

Maximum value in 'num_of_pairs' across all dfs:  9
Found in files:                                  ['20_data_with_zones.csv', '26_data_with_zones.csv']


## Create and fill TTC columns

1) add columns as many as max_value to each df 

 2) for non relevant rows (such as car rows) fill in with: "-99"

In [7]:
num_columns_to_add= max_value

In [8]:
#initialize ttc values, will be used when ttc is not present for peer

ttc_initial_value = float(-1) # this is garbage value for DL model to read

# Add columns as many as max_value to each df
for df in tqdm(dfs, desc="Processing DataFrames"):
    for i in range(1, int(num_columns_to_add) + 1):
        col_name = f'ttc_{i}'
        df[col_name] = ttc_initial_value #   # Initialize with NaN np.nan

    # Fill in TTC values for pedestrian rows
    for idx, row in df.iterrows():
        if row['class'] == 'pedestrian':
            ttc_lists = get_lists_of_lists_from_cell(row['pairs'])
            # Extract only the TTC values (assuming it's the third element in each sublist)
            ttc_values = [sublist[2] for sublist in ttc_lists if len(sublist) > 2]
            for i, ttc in enumerate(ttc_values):
                if i < num_columns_to_add:
                    df.at[idx, f'ttc_{i+1}'] = ttc

Processing DataFrames: 100%|██████████| 12/12 [00:23<00:00,  1.96s/it]


In [9]:
dfs[6]

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,xVelocity,yVelocity,class,...,zone_3,ttc_1,ttc_2,ttc_3,ttc_4,ttc_5,ttc_6,ttc_7,ttc_8,ttc_9
0,24,0,0,0,42.45444,-19.18280,299.87938,1.97658,-4.13336,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,24,0,1,1,42.84248,-19.99290,298.43114,1.78117,-3.82409,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,24,0,2,2,43.18715,-20.73831,296.83633,1.52714,-3.41870,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,24,0,3,3,43.46460,-21.38484,295.56309,1.25753,-2.95100,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,24,0,4,4,43.68598,-21.92451,294.65040,1.02239,-2.49450,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44706,24,422,4768,48,59.98829,-15.98387,16.86152,1.38540,0.48421,pedestrian,...,0.0,inf,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44707,24,422,4769,49,60.26446,-15.88487,18.15731,1.38401,0.47973,pedestrian,...,0.0,inf,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44708,24,422,4770,50,60.54115,-15.78903,19.39119,1.38998,0.45542,pedestrian,...,0.0,inf,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44709,24,422,4771,51,60.81861,-15.69779,20.59684,1.40749,0.41042,pedestrian,...,0.0,inf,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


DL models can get 'inf' as a number. so we need to replace all ttc 
'inf' valeus with a large number

In [10]:
value_to_replace_inf = 1111
# Replace inf values with a specific value
for df in dfs:
    df.replace([float('inf')], value_to_replace_inf, inplace=True)

In [11]:
dfs[6]

,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,xVelocity,yVelocity,class,...,zone_3,ttc_1,ttc_2,ttc_3,ttc_4,ttc_5,ttc_6,ttc_7,ttc_8,ttc_9
0,24,0,0,0,42.45444,-19.18280,299.87938,1.97658,-4.13336,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,24,0,1,1,42.84248,-19.99290,298.43114,1.78117,-3.82409,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,24,0,2,2,43.18715,-20.73831,296.83633,1.52714,-3.41870,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,24,0,3,3,43.46460,-21.38484,295.56309,1.25753,-2.95100,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,24,0,4,4,43.68598,-21.92451,294.65040,1.02239,-2.49450,car,...,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44706,24,422,4768,48,59.98829,-15.98387,16.86152,1.38540,0.48421,pedestrian,...,0.0,1111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44707,24,422,4769,49,60.26446,-15.88487,18.15731,1.38401,0.47973,pedestrian,...,0.0,1111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44708,24,422,4770,50,60.54115,-15.78903,19.39119,1.38998,0.45542,pedestrian,...,0.0,1111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
44709,24,422,4771,51,60.81861,-15.69779,20.59684,1.40749,0.41042,pedestrian,...,0.0,1111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


## move 'pairs' column

In [12]:
for df in dfs:
    #move num_of_pairs 
    column_to_move=df.pop("num_of_pairs")
    df.insert(15, "num_of_pairs", column_to_move)
    #move pairs
    column_to_move=df.pop("pairs")
    df.insert(15, "pairs", column_to_move)

In [13]:
# df.info()

## Create a track total duration (in frames) for each trackId

calculate how many frames (steps) are in each track

In [13]:
for df in dfs:
    # Get the maximum trackLifetime for each trackId
    track_durations = df.groupby('trackId')['trackLifetime'].max()
    track_durations=track_durations+1  # Add 1 to include the first '0' frame
    # Map the duration back to each row by trackId
    df['trackTotalNumFrames'] = df['trackId'].map(track_durations)
    # move to the 4th column
    column_to_move=df.pop("trackTotalNumFrames")
    df.insert(4, "trackTotalNumFrames", column_to_move)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67337 entries, 0 to 67336
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   recordingId          67337 non-null  int64  
 1   trackId              67337 non-null  int64  
 2   frame                67337 non-null  int64  
 3   trackLifetime        67337 non-null  int64  
 4   trackTotalNumFrames  67337 non-null  int64  
 5   xCenter              67337 non-null  float64
 6   yCenter              67337 non-null  float64
 7   heading              67337 non-null  float64
 8   xVelocity            67337 non-null  float64
 9   yVelocity            67337 non-null  float64
 10  class                67337 non-null  object 
 11  zone_zebra           67337 non-null  float64
 12  zone_1               67337 non-null  float64
 13  zone_2               67337 non-null  float64
 14  zone_3               67337 non-null  float64
 15  num_of_pairs         67337 non-null 

## Save to csv

In [15]:
file_name_suffix = "_data_flattened.csv"
file_name_data_list=save_dfs(root_dir, dfs, recordingId_list, folder_name, file_name_suffix)

Saving CSV files: 100%|██████████| 12/12 [00:05<00:00,  2.07it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\13_data_tcc_flattened
list of files saved:
['18_data_flattened.csv', '19_data_flattened.csv', '20_data_flattened.csv', '21_data_flattened.csv', '22_data_flattened.csv', '23_data_flattened.csv', '24_data_flattened.csv', '25_data_flattened.csv', '26_data_flattened.csv', '27_data_flattened.csv', '28_data_flattened.csv', '29_data_flattened.csv']
